In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
import math

n,m=symbols('n,m')
theta=symbols(r'\theta')
e1,e2,g12,nu12,nu21=symbols(r'E_1,E_2,G_{12},\nu_{12},\nu_{21}')

In [2]:
Ef=400*1e9
Em=2.5*1e9
vf=.7
vfiber=.7
vmatrix=.3
E1=Ef*vfiber+Em*vmatrix
E2=((vfiber/Ef)+(vmatrix/Em))**(-1)
Nu12=.24
Nu21=Nu12*(E2/E1)
G12=5.65*1e9
N=Matrix([[20.0],[20.0],[0.0]])*1e6
M=Matrix([[5.0],[5.0],[2.0]])*1e6
rf=(12/2)*1e-6#meters
rm=(rf**2/vfiber)**.5#meters
t=2.0*rm
zk=[[-4.0*t],[-3.0*t],[-2.0*t],[-t*1.0],[0.0],[t*1.0],[2.0*t],[3.0*t],[4.0*t]]#Matrix of layer edge z locations
Theta=Matrix([[45.0],[-45.0],[45.0],[-45.0],[45.0],[-45.0],[45.0],[-45.0]])#Matrix of theta locations
Qbarmat=Matrix([[e1/(1 - nu12*nu21), e2*nu12/(1 - nu12*nu21), 0], [e2*nu12/(1 - nu12*nu21), e2/(1 - nu12*nu21), 0], [0, 0, g12]])
#Theta=Matrix([[0.0],[90.0],[0.0],[90.0],[0.0],[90.0],[0.0],[90.0]])
#Theta=Matrix([[90.0],[45.0],[0.10],[-45.0],[-90.0],[-45.0],[45.0],[0.10]])
#Theta=Matrix([[90.0],[-90.0],[-90.0],[90.0],[-90.0],[90.0],[-90.0],[90.0]])
#thetaLinspace=np.linspace(-90,90,100)

$[\bar{Q}_k]=${{Qbarmat}}

In [3]:
def Qbar_k(theta):#function to determine Qbar_k for each k using theta
    M2=math.cos(math.radians(theta))
    N2=math.sin(math.radians(theta))
    T1=Matrix([[M2**2, N2**2, 2*M2*N2], [N2**2, M2**2, -2*M2*N2], [-M2*N2, M2*N2, M2**2 - N2**2]])
    T2=Matrix([[M2**2, N2**2, M2*N2], [N2**2, M2**2, -M2*N2], [-2*M2*N2, 2*M2*N2, M2**2 - N2**2]])
    Q=Matrix([[E1/(1 - Nu12*Nu21), E2*Nu12/(1 - Nu12*Nu21), 0], [E2*Nu12/(1 - Nu12*Nu21), E2/(1 - Nu12*Nu21), 0], [0, 0, G12]])
    Qbar=T1.inv()*Q*T2
    return Qbar

In [4]:
Qbark1=Qbar_k(Theta[0])#use function Qbar_k(Theta[i]) to find Qbar_k1
Qbark2=Qbar_k(Theta[1])
Qbark3=Qbar_k(Theta[2])
Qbark4=Qbar_k(Theta[3])
Qbark5=Qbar_k(Theta[4])
Qbark6=Qbar_k(Theta[5])
Qbark7=Qbar_k(Theta[6])
Qbark8=Qbar_k(Theta[7])
Qresults={"Qbark":[Qbark1,Qbark2,Qbark3,Qbark4,Qbark5,Qbark6,Qbark7,Qbark8]}

This gives,<br><br>
$[\bar{Q}]^{k=1}=${{Qbark1}}<br><br>
$[\bar{Q}]^{k=2}=${{Qbark2}}<br><br>
$[\bar{Q}]^{k=3}=${{Qbark3}}<br><br>
$[\bar{Q}]^{k=4}=${{Qbark4}}<br><br>
$[\bar{Q}]^{k=5}=${{Qbark5}}<br><br>
$[\bar{Q}]^{k=6}=${{Qbark6}}<br><br>
$[\bar{Q}]^{k=7}=${{Qbark7}}<br><br>
$[\bar{Q}]^{k=8}=${{Qbark8}}<br><br>

$[A] = \sum_{k=1}^{N} [\bar{Q_k}]^{k}(z_k-z_{k-1})$ <br><br>
$[B] = \frac{1}{2}\sum_{k=1}^{N} [\bar{Q_k}]^{k}(z_k^2-z_{k-1}^2)$ <br><br>
$[D] = \frac{1}{3}\sum_{k=1}^{N} [\bar{Q_k}]^{k}(z_k^3-z_{k-1}^3)$ <br><br>

In [12]:
def ABDmatrix(rf,Theta,zk):
    #rf is input in microns, theta in degrees, and zk in an array containing multiples of t eg Matrix([[-t],[0],[t]])
    
    A=0.0
    B=0.0
    D=0.0
    
    zk=np.array(zk)#make zk array data type
    
    ##Find A
    for i in range(0,len(Theta)):
        Aadd=Qbar_k(Theta[i])*(zk[i+1]-zk[i])#python is zero indexed zk[0] is the first element
        A+=Aadd
    A=A/(8*t)
    ##Find B
    for i in range(0,len(Theta)):
        Badd=Qbar_k(Theta[i])*(zk[i+1]**2-zk[i]**2)
        B+=Badd
    B=B/2.0
    B=B/(8*(t)**2)
    
    ##Find D
    for i in range(0,len(Theta)):
        Dadd=Qbar_k(Theta[i])*(zk[i+1]**3-zk[i]**3)
        D+=Dadd
    D=D/3.0
    D=D/(8*t**3)
    #Combine A, B, and D into a python dictionary data format variable
    ABD={"matrix":[A,B,D]}
    
    return ABD
    

In [13]:
ABD=ABDmatrix(rf,Theta,zk)#define ABD as the output of the function ABDmatrix

A=np.array(ABD["matrix"][0])#address dictionary with np.array(ABD["matrix"][i]) for data from ABD
B=np.array(ABD["matrix"][1])
D=np.array(ABD["matrix"][2])

This gives,<br><br>
$[A]=${{Matrix(A)}}<br><br>
$[B]=${{Matrix(B)}}<br><br>
$[D]=${{Matrix(D)}}<br><br>

$[A^{*}]=[A]^{-1}$<br><br>
$[B^{*}]=-[A]^{-1}[B]$<br><br> 
$[C^{*}]=[B][A]^{-1}$<br><br> 
$[D^{*}]=[D]-[B][A]^{-1}[B]$<br><br> 

In [14]:
def ABDstar(A,B,D):
    Astar=Matrix(A).inv()
    Bstar=np.matmul(-Astar,B)
    Cstar=np.matmul(B,Astar)
    Dstar=D-np.matmul(np.matmul(B,Astar),B)
    star_results={"star_matrices":[Astar,Bstar,Cstar,Dstar]}
    return star_results

In [15]:
Astar=ABDstar(A,B,D)["star_matrices"][0]
Bstar=ABDstar(A,B,D)["star_matrices"][1]
Cstar=ABDstar(A,B,D)["star_matrices"][2]
Dstar=ABDstar(A,B,D)["star_matrices"][3]

This gives,<br><br>
$[A^{*}]=${{Matrix(Astar)}}<br><br>
$[B^{*}]=${{Matrix(Bstar)}}<br><br>
$[C^{*}]=${{Matrix(Cstar)}}<br><br>
$[D^{*}]=${{Matrix(Dstar)}}<br><br>

{$\kappa$}=$-[D^{*}]^{-1}[C^{*}]${$N$}+$[D^{*}]^{-1}${$M$}

In [16]:
def kappa(Dstar,Cstar,N,M):
    kappa=np.matmul(np.matmul(-(np.array(Matrix(Dstar).inv())),Cstar),N)+np.matmul(Matrix(Dstar).inv(),M)
    return(kappa)

Kappa=kappa(Dstar,Cstar,N,M)

{$\epsilon^{o}$}=$[A^{*}]${$N$}$+[B^{*}]${$\kappa$}

In [17]:
def epsilon_not(Astar,N,Bstar,Kappa):
    epsilon_not=np.matmul(Astar,N)+np.matmul(Bstar,Kappa)
    return epsilon_not

Epsilon_not=epsilon_not(Astar,N,Bstar,Kappa)

{$\sigma$}$^{k}$=$[\bar{Q}]^{k}${$\epsilon^{o}$}+$[\bar{Q}]^{k}z${$\kappa$}

In [18]:
def sigma_k(Epsilon_not,Kappa,zk,Qresults):
    zk=np.array(zk)

    sigmak={"sigmak":[]}
    for i in range(0,len(Qresults["Qbark"])):
        sigmakadd=np.array(np.matmul(Qresults["Qbark"][i],Epsilon_not))+np.array(np.matmul(Qresults["Qbark"][i]*(zk[i]+(t/2)),Kappa))
        sigmak["sigmak"].append(sigmakadd)
    return sigmak

sigmak1=sigma_k(Epsilon_not,Kappa,zk,Qresults)["sigmak"][0]/1e6
sigmak2=sigma_k(Epsilon_not,Kappa,zk,Qresults)["sigmak"][1]/1e6
sigmak3=sigma_k(Epsilon_not,Kappa,zk,Qresults)["sigmak"][2]/1e6
sigmak4=sigma_k(Epsilon_not,Kappa,zk,Qresults)["sigmak"][3]/1e6
sigmak5=sigma_k(Epsilon_not,Kappa,zk,Qresults)["sigmak"][4]/1e6
sigmak6=sigma_k(Epsilon_not,Kappa,zk,Qresults)["sigmak"][5]/1e6
sigmak7=sigma_k(Epsilon_not,Kappa,zk,Qresults)["sigmak"][6]/1e6
sigmak8=sigma_k(Epsilon_not,Kappa,zk,Qresults)["sigmak"][7]/1e6

This gives,<br><br>
$[\sigma_{k}]^{k=1}=${{Matrix(sigmak1)}}Gpa<br><br>
$[\sigma_{k}]^{k=2}=${{Matrix(sigmak2)}}Gpa<br><br>
$[\sigma_{k}]^{k=3}=${{Matrix(sigmak3)}}Gpa<br><br>
$[\sigma_{k}]^{k=4}=${{Matrix(sigmak4)}}Gpa<br><br>
$[\sigma_{k}]^{k=5}=${{Matrix(sigmak5)}}Gpa<br><br>
$[\sigma_{k}]^{k=6}=${{Matrix(sigmak6)}}Gpa<br><br>
$[\sigma_{k}]^{k=7}=${{Matrix(sigmak7)}}Gpa<br><br>
$[\sigma_{k}]^{k=8}=${{Matrix(sigmak8)}}Gpa<br><br>